# DOWNLOAD MISSING RETWEET DATA

In [2]:
import tweepy
import csv
import pandas as pd
import psycopg2

# Connect to DB

consumer_key = 'KfMKNmydahwQSubsT297DpbYt'
consumer_secret = 'ccCdXjEEM1vPsuA4sgJNAOTwW9EueZHejhBe42VjO2brWexLdt'
access_token = '819912293521772548-hE5XFuCPGIh5K6GWKp541zwf06YJ4sg'
access_token_secret = 'RQahNZKvkS5iVFr5bKF1PI9HpW6yxmhWyPI5weGKU5yaR'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

try:
    conn = psycopg2.connect("dbname=eksploracja_danych user=postgres password=postgres", host="localhost")
    print "Connected to the database"
except:
    print "I am unable to connect to the database"
    
cur = conn.cursor()

# Try on connection

try:
    cur.execute("""SELECT id, phrase FROM hashtag""")
    cur.execute("""SELECT author_id, tweet_id, date FROM re_tweet""")
    cur.execute("""SELECT id, name FROM author""")
    cur.execute("""SELECT id, author_id, searched_hashtag_id, date, content from tweet""")
    print "Properly set all tables"
except:
    print "I can't properly commit transactions to DB"

Connected to the database
Properly set all tables


/usr/local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
update_retweet_details = """ UPDATE re_tweet 
SET 
source_device = %s, 
in_reply_to_status_id = %s, 
in_reply_to_user_id = %s, 
in_reply_to_screen_name = %s, 
retweet_count = %s, 
favorite_count = %s, 
retweeted = %s, 
retweeted_tweet_id = %s, 
retweeted_tweet_author = %s, 
content = %s 
WHERE 
tweet_id = %s;
"""

In [13]:
# TEST RUN
status = api.get_status(1056861827567628288)

print status.id
print status.text
print status.in_reply_to_status_id
print status.in_reply_to_user_id
print status.in_reply_to_screen_name
print status.retweet_count
print status.favorite_count
print status.retweeted
print status.user.id
try:
    print status.retweeted_status.id
    print status.retweeted_status.user.id
except:
    print 'None'
    print 'None'

try:
    retweeted_tweet_id = status.retweeted_status.id
    retweeted_tweet_author =  status.retweeted_status.user.id
except:
    retweeted_tweet_id = None
    retweeted_tweet_author =  None

cur.execute(update_retweet_details, (status.source, 
                                    status.in_reply_to_status_id, 
                                    status.in_reply_to_user_id,
                                    status.in_reply_to_screen_name,
                                    status.retweet_count,
                                    status.favorite_count,
                                    status.retweeted,
                                    retweeted_tweet_id, 
                                    retweeted_tweet_author,
                                    status.text,
                                    status.id
                                   ))
conn.commit()

TweepError: [{u'message': u'No status found with that ID.', u'code': 144}]

In [6]:
# GET ALL IDS OF TWEETS TO UPDATE

def zmien(n):
    return int(n[0])

cur.execute("""SELECT tweet_id FROM re_tweet""")
ids = map(zmien, cur.fetchall())
print len(ids)

31938


In [7]:
# METHOD FOR DOWNLOADING DATA ABOUT STATusES FOR PROVIDED IDS

def lookup_tweets_list(tweets_id, api):
    full_tweets = []
    tweets_count = len(tweets_id)
    while True:
        try:
            for i in range((tweets_count / 100) + 1):
                full_tweets.extend(api.statuses_lookup(tweets_id[i*100:min((i+1)*100, tweets_count)]))
                print 'getting tweets batch:', i
        except tweepy.TweepError as e:
            print 'Something went wrong, quitting...', e
            time.sleep(15 * 60)
        return full_tweets

In [10]:
# DWONLOADING STATUSES DATA

print "Starting download.."
tweets = lookup_tweets_list(ids, api)

print "inserting..."
for status in tweets:
    try:
        retweeted_tweet_id = status.retweeted_status.id
        retweeted_tweet_author =  status.retweeted_status.user.id
    except:
        retweeted_tweet_id = None
        retweeted_tweet_author =  None
    
    cur.execute(update_retweet_details, (status.source, 
                                        status.in_reply_to_status_id, 
                                        status.in_reply_to_user_id,
                                        status.in_reply_to_screen_name,
                                        status.retweet_count,
                                        status.favorite_count,
                                        status.retweeted,
                                        retweeted_tweet_id, 
                                        retweeted_tweet_author,
                                        status.text,
                                        status.id
                                       ))
    conn.commit()   

Starting download..
getting tweets batch: 0
getting tweets batch: 1
getting tweets batch: 2
getting tweets batch: 3
getting tweets batch: 4
getting tweets batch: 5
getting tweets batch: 6
getting tweets batch: 7
getting tweets batch: 8
getting tweets batch: 9
getting tweets batch: 10
getting tweets batch: 11
getting tweets batch: 12
getting tweets batch: 13
getting tweets batch: 14
getting tweets batch: 15
getting tweets batch: 16
getting tweets batch: 17
getting tweets batch: 18
getting tweets batch: 19
getting tweets batch: 20
getting tweets batch: 21
getting tweets batch: 22
getting tweets batch: 23
getting tweets batch: 24
getting tweets batch: 25
getting tweets batch: 26
getting tweets batch: 27
getting tweets batch: 28
getting tweets batch: 29
getting tweets batch: 30
getting tweets batch: 31
getting tweets batch: 32
getting tweets batch: 33
getting tweets batch: 34
getting tweets batch: 35
getting tweets batch: 36
getting tweets batch: 37
getting tweets batch: 38
getting tweets 

getting tweets batch: 319
inserting...
